In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

INPUTS

In [43]:
# a parede é um cubo/quadrado determinado pelo tamanho do lado do cubo/quadrado e numero de dimensoes
tamanho_parede = 10
numero_dimensoes = 3
quantidade_particulas = 10
raio = 1
massa = 1
velocidade_maxima = 10000

PARTICULAS

- Classe das particulas

In [94]:
class Particula:
    def __init__(self, raio: float, massa: float, vetor_posicao: float, vetor_velocidade: float) -> None:
        self.massa = massa
        self.raio = raio
        self.vetor_posicao = vetor_posicao
        self.vetor_velocidade = vetor_velocidade

    @property
    def momento(self):
        return self.massa * self.vetor_velocidade
    
    @property
    def energia_cinetica(self):
        return self.massa * self.vetor_velocidade * self.vetor_velocidade
    
    def atualizar_posicao(self, tempo_decorrido: float) -> None:
        self.vetor_posicao += self.vetor_velocidade * tempo_decorrido
    
    def parede_reflexao(self, dimensao_da_parede: int) -> None:
        self.vetor_velocidade[dimensao_da_parede] = - self.vetor_velocidade[dimensao_da_parede]

    def __str__(self) -> str:
        return "Raio {}, massa {}, posicao {} e velocidade {}".format(self.raio, self.massa, self.vetor_posicao, self.vetor_velocidade)
    
    def __repr__(self) -> str:
        return "Particula({}, {}, np.array({}), np.array({}))".format(self.raio, self.massa, self.vetor_posicao.tolist(), self.vetor_velocidade.tolist())

- RNG
- Criador de vetor aleatorio (com dimensoes certas e sem zeros)
- Criador de particula aleatoria

In [84]:
# o RNG é um gerador aleatorio de numeros (float) entre [0, 1)
rng = np.random.default_rng()

# gerador de vetor aleatorio (sem zeros)
# nenhuma posicao inicial pode ser 0 pra nao ter problemas com as paredes, entao esse gerador remapeia zeros
def vetor_aleatorio(valor_minimo: float, valor_maximo: float):
    vetor_zero_a_um = rng.random(numero_dimensoes)
    
    while min(vetor_zero_a_um) == 0:
        vetor_zero_a_um[vetor_zero_a_um == 0] = rng.random() # remapeia valores que forem zero

    vetor_aleatorio = (valor_maximo - valor_minimo) * vetor_zero_a_um + valor_minimo
    return vetor_aleatorio

# criador de particula
def criador_de_particula(raio: float, massa: float) -> Particula:
    posicao_criada = vetor_aleatorio(0, tamanho_parede)
    velocidade_criada = vetor_aleatorio(-velocidade_maxima, velocidade_maxima) # velocidades podem ser positivas ou negativas

    particula_criada = Particula(raio, massa, posicao_criada, velocidade_criada)
    return particula_criada

CRIADOR DE LISTA DE PARTICULAS

- Calculador de momento total de uma lista de particulas

In [ ]:
def momento_total_lista(lista_de_particulas: list[Particula]):
    momento_total = np.zeros(numero_dimensoes)
    for particula in lista_de_particulas:
        momento_total += particula.momento
        
    return momento_total

- Criador da lista

In [16]:
def criar_particulas_iniciais(quantidade_particulas: int, raio: float, massa: float) -> list[Particula]: 
    # primeiro monta uma lista inicial das particulas
    lista_particulas = [criador_de_particula(raio, massa) for i in range(quantidade_particulas)]

    # depois calculamos o momento total das particulas (exceto a ultima)
    momento_total = momento_total_lista(lista_particulas[:-1])
    
    # e fazemos as velocidades da ultima particula equilibrar para o momento total ser zero 
    ultima_particula = lista_particulas[-1]
    velocidade_ultima_particula = - momento_total / ultima_particula.massa
    ultima_particula.vetor_velocidade = velocidade_ultima_particula

    return lista_particulas
        

TESTAGEM

In [95]:
teste = Particula(1,2,np.array([1,2,3]),np.array([5,10,15]))

In [32]:
print(teste.vetor_posicao)
teste.atualizar_posicao(1)
print(teste.vetor_posicao)

[1 2 3]
[ 6 12 18]


In [33]:
print(teste.vetor_velocidade)
teste.parede_reflexao(2)
print(teste.vetor_velocidade)

[ 5 10 15]
[  5  10 -15]
